In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data.sampler import SubsetRandomSampler
import os
import numpy as np
from tqdm import tqdm

In [2]:
lr = 0.01
wd = 1e-6
ne = 60
nsc = 20
gamma = 0.32
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#To get reproductible experiment
torch.manual_seed(0)
np.random.seed(0)

In [3]:
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.2, 0.2, 0.2)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)
# Création de l'ensemble de validation
validset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_test)
split=5000
num_train=50000
indices = list(range(num_train))
np.random.seed(123)
np.random.shuffle(indices)
train_idx, valid_idx = indices[split:], indices[:split]
trainset = torch.utils.data.Subset(trainset,train_idx)
validset = torch.utils.data.Subset(validset,valid_idx)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(validset, batch_size=100, shuffle=False, num_workers=2)
#Création de l'ensemble de test
testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader = torch.utils.data.DataLoader(testset, batch_size=100, shuffle=False, num_workers=2)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified
Files already downloaded and verified


In [4]:
architecture = [64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M']
class VGG(nn.Module):
    def __init__(self,architecture):
        super(VGG, self).__init__()
        self.layers = self.make_layers(architecture)
        self.classifier = nn.Linear(512, 10)
    def forward(self,x):
        x = self.layers(x)
        x = x.view(x.size(0),-1)
        return self.classifier(x)
    def make_layers(self,architecture):
        layers = []
        in_channels = 3
        for x in architecture:
            if x=='M':
                layers += [nn.MaxPool2d(kernel_size=[3,3], stride=(1,1))]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=[3, 3], stride=(1, 1), padding=(1, 1)),
                           nn.BatchNorm2d(x),
                           nn.ReLU()]
                in_channels = x
        return nn.Sequential(*layers)
            


net = VGG(architecture)





net = net.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=lr, momentum=0.9, weight_decay=wd)
lr_sc = lr_scheduler.StepLR(optimizer, step_size=nsc, gamma=gamma)
# Training
def train(epoch,trainloader):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    loop = tqdm(enumerate(trainloader), total=len(trainloader))
    for batch_idx, (inputs, targets) in loop:
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        loop.set_description(f"Epoch [{epoch}]")
        loop.set_postfix(acc=correct/total)

def test(epoch,validloader):
    global best_acc
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        loop = tqdm(enumerate(validloader), total=len(validloader))
        for batch_idx, (inputs, targets) in loop:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loop.set_postfix(acc=correct/total)
    # Save checkpoint.
    acc = 100.*correct/total
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': net.state_dict(),
            'acc': acc,
            'epoch': epoch,
        }
        if not os.path.isdir('checkpoint'):
            os.mkdir('checkpoint')
        torch.save(state, './checkpoint/ckpt.t7')
        best_acc = acc
        
def testfinal(testloader):
    print('==> Resuming from checkpoint..')
    assert os.path.isdir('checkpoint'), 'Error: no checkpoint directory found!'
    checkpoint = torch.load('./checkpoint/ckpt.t7')
    net.load_state_dict(checkpoint['net'])
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        loop = tqdm(enumerate(testloader), total=len(testloader))
        for batch_idx, (inputs, targets) in loop:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
            loop.set_postfix(acc=correct/total)
        print("\nFinal accuracy on the test set : ",correct/total)

best_acc = 0       

for epoch in range(0, ne):
    train(epoch,trainloader)
    test(epoch,validloader)
    lr_sc.step()

testfinal(testloader)


100%|██████████| 50/50 [00:02<00:00, 17.91it/s, acc=0.544]


Saving..


100%|██████████| 50/50 [00:02<00:00, 17.77it/s, acc=0.68]


Saving..


100%|██████████| 50/50 [00:02<00:00, 17.92it/s, acc=0.731]


...

Saving..


100%|██████████| 50/50 [00:02<00:00, 17.59it/s, acc=0.912]


==> Resuming from checkpoint..


100%|██████████| 100/100 [00:05<00:00, 18.08it/s, acc=0.909]


Final accuracy on the test set :  0.9092
